In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

import urllib
from PIL import Image
import glob

In [4]:
PATH = 'data/face/'
image_folder = f'{PATH}train/'
data_csv = PATH+'tw_dem_images.csv'
image_csv = PATH+'image_csv.csv'

### Load Data

In [5]:
# df = pd.read_csv(PATH+'Photo_Income_Age_2.8.18.csv')
df = pd.read_csv(data_csv, dtype={'local_photo': 'object', 'image_type': 'object'})

In [ ]:
uid = 923214
row = df.loc[df.userID == uid].iloc[0]

In [ ]:
row

In [ ]:
df.loc[row.name]
df.at[row.name, 'image_type'] = 'corrupted'

### Explore Data

In [ ]:
df.columns

In [ ]:
df['userID'].size

In [ ]:
df.head()

In [ ]:
df.loc[df.userID == 922512].image_type.isna()

### Recreating dataset with only the face cropped images

### Create Image Frame with only JPEGS

In [10]:
img_df = df.loc[(df.local_photo != '') 
                & (df.local_photo != 'corrupted') 
                & (df.image_type == 'jpeg') 
                & ~df.local_photo.isna()
               ]


In [11]:
training_files = glob.glob(image_folder+'*.jpg')

In [52]:
file_ids = list(map(lambda x: int(x[len(image_folder):-len('.jpg')]), training_files))

In [53]:
img_df.head()

,gender,attractedToGender,fromState,metro_name,User_Age,User_Photo,User_Income,local_photo,image_type
userID,,,,,,,,,
992835,female,male,NC,NaN,43.0,https://s3-us-west-1.amazonaws.com/tawkifyfile...,0.0,data/tawkify/train/992835.jpg,jpeg
992831,female,male,OH,NaN,40.0,https://s3-us-west-1.amazonaws.com/tawkifyfile...,40000.0,data/tawkify/train/992831.jpg,jpeg
992820,male,female,TX,NaN,49.0,https://s3-us-west-1.amazonaws.com/tawkifyfile...,40000.0,data/tawkify/train/992820.jpg,jpeg
992814,male,female,WA,NaN,29.0,https://s3-us-west-1.amazonaws.com/tawkifyfile...,0.0,data/tawkify/train/992814.jpg,jpeg
992813,female,male,SC,NaN,46.0,https://s3-us-west-1.amazonaws.com/tawkifyfile...,0.0,data/tawkify/train/992813.jpg,jpeg


In [54]:
img_df.index

Int64Index([992835, 992831, 992820, 992814, 992813, 992810, 992807, 992797,
            992796, 992788,
            ...
              7104,   6835,   6525,   4814,   4035,   3679,   2286,    971,
               934,    592],
           dtype='int64', name='userID', length=89644)

In [55]:
face_df = img_df[img_df.index.isin(list(file_ids))]

In [59]:
img_df.shape

(89644, 9)

In [60]:
face_df.shape

(58642, 9)

In [61]:
face_df.to_csv(image_csv)

In [33]:
img_df = pd.read_csv(image_csv, index_col='userID')

### Facial detection pipeline - 

https://hackernoon.com/building-a-facial-recognition-pipeline-with-deep-learning-in-tensorflow-66e7645015b8

In [5]:
# Copyright 2015-2016 Carnegie Mellon University
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Module for dlib-based alignment."""

# NOTE: This file has been copied from the openface project.
#  https://github.com/cmusatyalab/openface/blob/master/openface/align_dlib.py

import cv2
import dlib
import numpy as np

TEMPLATE = np.float32([
    (0.0792396913815, 0.339223741112), (0.0829219487236, 0.456955367943),
    (0.0967927109165, 0.575648016728), (0.122141515615, 0.691921601066),
    (0.168687863544, 0.800341263616), (0.239789390707, 0.895732504778),
    (0.325662452515, 0.977068762493), (0.422318282013, 1.04329000149),
    (0.531777802068, 1.06080371126), (0.641296298053, 1.03981924107),
    (0.738105872266, 0.972268833998), (0.824444363295, 0.889624082279),
    (0.894792677532, 0.792494155836), (0.939395486253, 0.681546643421),
    (0.96111933829, 0.562238253072), (0.970579841181, 0.441758925744),
    (0.971193274221, 0.322118743967), (0.163846223133, 0.249151738053),
    (0.21780354657, 0.204255863861), (0.291299351124, 0.192367318323),
    (0.367460241458, 0.203582210627), (0.4392945113, 0.233135599851),
    (0.586445962425, 0.228141644834), (0.660152671635, 0.195923841854),
    (0.737466449096, 0.182360984545), (0.813236546239, 0.192828009114),
    (0.8707571886, 0.235293377042), (0.51534533827, 0.31863546193),
    (0.516221448289, 0.396200446263), (0.517118861835, 0.473797687758),
    (0.51816430343, 0.553157797772), (0.433701156035, 0.604054457668),
    (0.475501237769, 0.62076344024), (0.520712933176, 0.634268222208),
    (0.565874114041, 0.618796581487), (0.607054002672, 0.60157671656),
    (0.252418718401, 0.331052263829), (0.298663015648, 0.302646354002),
    (0.355749724218, 0.303020650651), (0.403718978315, 0.33867711083),
    (0.352507175597, 0.349987615384), (0.296791759886, 0.350478978225),
    (0.631326076346, 0.334136672344), (0.679073381078, 0.29645404267),
    (0.73597236153, 0.294721285802), (0.782865376271, 0.321305281656),
    (0.740312274764, 0.341849376713), (0.68499850091, 0.343734332172),
    (0.353167761422, 0.746189164237), (0.414587777921, 0.719053835073),
    (0.477677654595, 0.706835892494), (0.522732900812, 0.717092275768),
    (0.569832064287, 0.705414478982), (0.635195811927, 0.71565572516),
    (0.69951672331, 0.739419187253), (0.639447159575, 0.805236879972),
    (0.576410514055, 0.835436670169), (0.525398405766, 0.841706377792),
    (0.47641545769, 0.837505914975), (0.41379548902, 0.810045601727),
    (0.380084785646, 0.749979603086), (0.477955996282, 0.74513234612),
    (0.523389793327, 0.748924302636), (0.571057789237, 0.74332894691),
    (0.672409137852, 0.744177032192), (0.572539621444, 0.776609286626),
    (0.5240106503, 0.783370783245), (0.477561227414, 0.778476346951)])

INV_TEMPLATE = np.float32([
    (-0.04099179660567834, -0.008425234314031194, 2.575498465013183),
    (0.04062510634554352, -0.009678089746831375, -1.2534351452524177),
    (0.0003666902601348179, 0.01810332406086298, -0.32206331976076663)])

TPL_MIN, TPL_MAX = np.min(TEMPLATE, axis=0), np.max(TEMPLATE, axis=0)
MINMAX_TEMPLATE = (TEMPLATE - TPL_MIN) / (TPL_MAX - TPL_MIN)


class AlignDlib:
    """
    Use `dlib's landmark estimation <http://blog.dlib.net/2014/08/real-time-face-pose-estimation.html>`_ to align faces.
    The alignment preprocess faces for input into a neural network.
    Faces are resized to the same size (such as 96x96) and transformed
    to make landmarks (such as the eyes and nose) appear at the same
    location on every image.
    Normalized landmarks:
    .. image:: ../images/dlib-landmark-mean.png
    """

    #: Landmark indices corresponding to the inner eyes and bottom lip.
    INNER_EYES_AND_BOTTOM_LIP = [39, 42, 57]

    #: Landmark indices corresponding to the outer eyes and nose.
    OUTER_EYES_AND_NOSE = [36, 45, 33]

    def __init__(self, facePredictor):
        """
        Instantiate an 'AlignDlib' object.
        :param facePredictor: The path to dlib's
        :type facePredictor: str
        """
        assert facePredictor is not None

        # pylint: disable=no-member
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(facePredictor)

    def getAllFaceBoundingBoxes(self, rgbImg):
        """
        Find all face bounding boxes in an image.
        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :return: All face bounding boxes in an image.
        :rtype: dlib.rectangles
        """
        assert rgbImg is not None

        try:
            return self.detector(rgbImg, 1)
        except Exception as e:  # pylint: disable=broad-except
            print("Warning: {}".format(e))
            # In rare cases, exceptions are thrown.
            return []

    def getLargestFaceBoundingBox(self, rgbImg, skipMulti=False):
        """
        Find the largest face bounding box in an image.
        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :param skipMulti: Skip image if more than one face detected.
        :type skipMulti: bool
        :return: The largest face bounding box in an image, or None.
        :rtype: dlib.rectangle
        """
        assert rgbImg is not None

        faces = self.getAllFaceBoundingBoxes(rgbImg)
        if (not skipMulti and len(faces) > 0) or len(faces) == 1:
            return max(faces, key=lambda rect: rect.width() * rect.height())
        else:
            return None
        
    def get_crop_rect(self, rgbImg, face_rect):
        img_h, img_w, _ = rgbImg.shape
        center = face_rect.center()
        wx2 = face_rect.width() * 2
        left = (center.x - wx2)
        right = (center.x + wx2)
        top = (center.y - wx2)
        bottom = (center.y + wx2)

        # if not enough space
        # left < 0
        # right > w
        # top < 0
        # bottom > h

        # dim - direction
        # left = 0 + left(-7) = -7
        # right = 200 - right(220) = -20
        # top = 0 + top(-23) = -23
        # bottom = 300 - bottom(330) = -30

        min_dist = min(left, img_w-right, top, img_h-bottom, 0) # min should be 0 if all else positive.
        new_wx2 = face_rect.width()*2 + min_dist


        left = (center.x - new_wx2)
        right = (center.x + new_wx2)
        top = (center.y - new_wx2)
        bottom = (center.y + new_wx2)
        return left, right, top, bottom

    def face_crop(self, rgbImg):
        bounding_box = self.getLargestFaceBoundingBox(rgbImg, skipMulti=True)
        if bounding_box is None or bounding_box.width() < 100: # emojii detection case...
            return None
        l,r,t,b = self.get_crop_rect(rgbImg, bounding_box)
        
        return rgbImg[t:b, l:r, :]
        

    def findLandmarks(self, rgbImg, bb):
        """
        Find the landmarks of a face.
        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :param bb: Bounding box around the face to find landmarks for.
        :type bb: dlib.rectangle
        :return: Detected landmark locations.
        :rtype: list of (x,y) tuples
        """
        assert rgbImg is not None
        assert bb is not None

        points = self.predictor(rgbImg, bb)
        # return list(map(lambda p: (p.x, p.y), points.parts()))
        return [(p.x, p.y) for p in points.parts()]

    # pylint: disable=dangerous-default-value
    def align(self, imgDim, rgbImg, bb=None,
              landmarks=None, landmarkIndices=INNER_EYES_AND_BOTTOM_LIP,
              skipMulti=False, scale=1.0):
        r"""align(imgDim, rgbImg, bb=None, landmarks=None, landmarkIndices=INNER_EYES_AND_BOTTOM_LIP)
        Transform and align a face in an image.
        :param imgDim: The edge length in pixels of the square the image is resized to.
        :type imgDim: int
        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :param bb: Bounding box around the face to align. \
                   Defaults to the largest face.
        :type bb: dlib.rectangle
        :param landmarks: Detected landmark locations. \
                          Landmarks found on `bb` if not provided.
        :type landmarks: list of (x,y) tuples
        :param landmarkIndices: The indices to transform to.
        :type landmarkIndices: list of ints
        :param skipMulti: Skip image if more than one face detected.
        :type skipMulti: bool
        :param scale: Scale image before cropping to the size given by imgDim.
        :type scale: float
        :return: The aligned RGB image. Shape: (imgDim, imgDim, 3)
        :rtype: numpy.ndarray
        """
        assert imgDim is not None
        assert rgbImg is not None
        assert landmarkIndices is not None

        if bb is None:
            bb = self.getLargestFaceBoundingBox(rgbImg, skipMulti)
            if bb is None:
                return

        if landmarks is None:
            landmarks = self.findLandmarks(rgbImg, bb)

        npLandmarks = np.float32(landmarks)
        npLandmarkIndices = np.array(landmarkIndices)

        # pylint: disable=maybe-no-member
        H = cv2.getAffineTransform(npLandmarks[npLandmarkIndices],
                                   imgDim * MINMAX_TEMPLATE[npLandmarkIndices] * scale + imgDim * (1 - scale) / 2)
        thumbnail = cv2.warpAffine(rgbImg, H, (imgDim, imgDim))

        return thumbnail

In [6]:
import argparse
import glob
import logging
import multiprocessing as mp
import os
import time

import cv2

# from medium_facenet_tutorial.align_dlib import AlignDlib

logger = logging.getLogger(__name__)

align_dlib = AlignDlib(f'{PATH}shape_predictor_68_face_landmarks.dat')


def preprocess_image(input_path, output_path, crop_dim):
    """
    Detect face, align and crop :param input_path. Write output to :param output_path
    :param input_path: Path to input image
    :param output_path: Path to write processed image
    :param crop_dim: dimensions to crop image to
    """
    image = _process_image(input_path, crop_dim)
    if image is not None:
        logger.debug('Writing processed file: {}'.format(output_path))
        cv2.imwrite(output_path, image)
    else:
        logger.warning("Skipping filename: {}".format(input_path))


def _process_image(filename, crop_dim):
    image = None
    aligned_image = None
    image = _buffer_image(filename)
    if image is not None:
#         aligned_image = _align_image(image, crop_dim)
        cropped_image = _face_crop(image)
    else:
        raise IOError('Error buffering image: {}'.format(filename))

    return cropped_image


def _buffer_image(filename):
    logger.debug('Reading image: {}'.format(filename))
    image = cv2.imread(filename, )
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def _face_crop(image):
    face_crop = align_dlib.face_crop(image)
#     if face_crop is not None:
#         face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
    return face_crop
    

def _align_image(image, crop_dim):
    bb = align_dlib.getLargestFaceBoundingBox(image)
    aligned = align_dlib.align(crop_dim, image, bb, landmarkIndices=AlignDlib.INNER_EYES_AND_BOTTOM_LIP)
    if aligned is not None:
        aligned = cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB)
    return aligned


# if __name__ == '__main__':
#     logging.basicConfig(level=logging.INFO)
#     parser = argparse.ArgumentParser(add_help=True)
#     parser.add_argument('--input-dir', type=str, action='store', default='data', dest='input_dir')
#     parser.add_argument('--output-dir', type=str, action='store', default='output', dest='output_dir')
#     parser.add_argument('--crop-dim', type=int, action='store', default=180, dest='crop_dim',
#                         help='Size to crop images to')

#     args = parser.parse_args()

#     main(args.input_dir, args.output_dir, args.crop_dim)

In [7]:
def test_process(input_file):
    output_file = input_file.replace('train', 'output')
    plt.figure()
    plt.imshow(plt.imread(input_file))
    preprocess_image(input_file, output_file, 200)
    plt.figure()
    plt.imshow(plt.imread(output_file))

In [ ]:
input_files = glob.glob(f'{PATH}train/*')

In [ ]:
test_process(input_files[50])

### Process image

In [ ]:
# def process(input_dir, output_dir, crop_dim):
#     start_time = time.time()
#     pool = mp.Pool(processes=mp.cpu_count())

#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for image_dir in os.listdir(input_dir):
#         image_output_dir = os.path.join(output_dir, os.path.basename(os.path.basename(image_dir)))
#         if not os.path.exists(image_output_dir):
#             os.makedirs(image_output_dir)

#     image_paths = glob.glob(os.path.join(input_dir, '**/*.jpg'))
#     for index, image_path in enumerate(image_paths):
#         image_output_dir = os.path.join(output_dir, os.path.basename(os.path.dirname(image_path)))
#         output_path = os.path.join(image_output_dir, os.path.basename(image_path))
#         pool.apply_async(preprocess_image, (image_path, output_path, crop_dim))

#     pool.close()
#     pool.join()
#     logger.info('Completed in {} seconds'.format(time.time() - start_time))



In [7]:
def process(input_dir, output_dir, crop_dim):
    start_time = time.time()
    pool = mp.Pool(processes=mp.cpu_count())

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_paths = glob.glob(os.path.join(input_dir, '*.jpg'))
    for index, image_path in enumerate(image_paths):
        output_path = image_path.replace(input_dir, output_dir)
        pool.apply_async(preprocess_image, (image_path, output_path, crop_dim))

    pool.close()
    pool.join()
    logger.info('Completed in {} seconds'.format(time.time() - start_time))



In [ ]:
process('data/tawkify/train/', f'{PATH}train/', 200)

In [ ]:
image = cv2.imread(input_files[12])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
bbxs = align_dlib.getAllFaceBoundingBoxes(image)
fbx = bbxs[0]

In [ ]:
input_files[12]

In [ ]:
image.shape

In [ ]:

# wx2 = (fbx.width() * 2)
# left = max(center.x - wx2, 10)
# right = min(center.x + wx2, image.shape[1]-10)
# top = max(center.y - int(fbx.height()*2), 10)
# bottom = min(center.y + int(fbx.height() * 2), image.shape[0]-10)

### Cropping alg

In [ ]:
def get_crop_rect(image, face_rect):
    img_h, img_w, _ = image.shape
    center = fbx.center()
    wx2 = fbx.width() * 2
    left = (center.x - wx2)
    right = (center.x + wx2)
    top = (center.y - wx2)
    bottom = (center.y + wx2)
    
    # if not enough space
    # left < 0
    # right > w
    # top < 0
    # bottom > h

    # dim - direction
    # left = 0 + left(-7) = -7
    # right = 200 - right(220) = -20
    # top = 0 + top(-23) = -23
    # bottom = 300 - bottom(330) = -30

    min_dist = min(left, img_w-right, top, img_h-bottom, 0) # min should be 0 if all else positive.
    new_wx2 = fbx.width()*2 + min_dist


    left = (center.x - new_wx2)
    right = (center.x + new_wx2)
    top = (center.y - new_wx2)
    bottom = (center.y + new_wx2)
    return left, right, top, bottom, (left, top), (right, bottom)


In [ ]:
# cv2.rectangle(image,(fbx.left(),fbx.top()),(fbx.right(),fbx.bottom()),(0,255,0),2)
l,r,t,b = get_crop_rect(image, fbx)
img_bx = cv2.rectangle(image,(l,t),(r,b),(0,255,0),10)

In [ ]:
plt.imshow(img_bx)

In [ ]:
cropped_img = image[t:b, l:r, :]

In [ ]:
plt.imshow(cropped_img)

Edge cases:  
'data/face/train/957569.jpg' -> input_files[12] -> emojii
